### regional prices and timespreads

In [10]:
##### this is it
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib
import matplotlib.pyplot as plt
import requests
import sqlite3
import datetime
import time
import os
import json
import ciso8601 #to parse datetime to timestamp and then string 
# import unirest
from numpy import cos, sin, arcsin, sqrt
from math import radians
import http.client
from dateutil.parser import parse
# from geopy import distance

In [11]:
start_year = 2015                   # Bloomberg data starts from 2011
start_date = datetime.date(year=start_year,month=1,day=1)                       # start date for sending queries
time_range = 7                      #days - for max query range for airports
today = datetime.datetime.utcnow()  #database is based on UTC
today = today.date()
end_date = today                    # can be anything
global_index = pd.date_range(start=start_date,end=end_date, freq='W-FRI')       # create weekly datetime_index ending this/
# not including crude for this model at the moment
# expiry calendar needs to be incorporated into the model
# look at contango etc

In [12]:
## for bloomberg data
# all the prices from bbg is in here
bbg_all = pd.read_csv('bbg_weekly_all.csv',header=0,index_col=0)                # dates are in terrible format
temp_index = bbg_all.index.to_list()                                        # need to fix the index to datetime
dates_list = [datetime.datetime.strptime(x, '%m/%d/%Y') for x in temp_index]    # change every element from str to datetimeindex
bbg_all['week_ending'] = pd.to_datetime(dates_list)
bbg_all = bbg_all.set_index('week_ending')
# now the selected prices tickers FOR MEa
bbg_my = pd.read_csv('bloomberg_tickers.csv',index_col=0)       # this is the meta for all the tickers  # dates are in 
bbg_my_index = bbg_my.index.to_list()
bbg_tix = bbg_all.columns.to_list()
refrnce_df = pd.DataFrame(index=[bbg_my_index],columns=['long_name','short_name'])
### SELECT THE SUBSET WITH MY TICKERS!! BUT INCLUDES CRUDE bbg_my_index
MY_BBG = bbg_all[bbg_all.columns.intersection(bbg_my_index)]
# aMY_BBG_1 = bbg_all[bbg_all.columns.intersection(bbg_my_index)]
# JET 
LA_JET = 'FSLJM'
GC_JET = 'FSGJM'
NY_JET = 'FSNJM'
# HO is the only disty. No data for LA and NY, oddly
CMDTY = 'Comdty'
#SPRD JET-DISTY
LA_SPRD = 'FLJSM'
GC_SPRD = 'FGJSM'
NY_SPRD = 'FNJSM'

In [13]:
###### JET #####
# ny
NY_JET = MY_BBG.filter(regex=NY_JET)
NY_JET_cols = NY_JET.columns.to_list()
NY_JET_COLS = ['NY_JET_1','NY_JET_2','NY_JET_3']
NEW_NY = dict(zip(NY_JET_cols, NY_JET_COLS))
NY_JET.rename(columns=NEW_NY,inplace=True)
NY_JET['M1_M2'] = NY_JET['NY_JET_1'] - NY_JET['NY_JET_2']
NY_JET['M1_M3'] = NY_JET['NY_JET_1'] - NY_JET['NY_JET_3']
NY_JET['M2_M3'] = NY_JET['NY_JET_2'] - NY_JET['NY_JET_3']


In [14]:
###### JET #####
# gc
GC_JET = MY_BBG.filter(regex=GC_JET)
GC_JET_cols = GC_JET.columns.to_list()
GC_JET_COLS = ['GC_JET_1','GC_JET_2','GC_JET_3']
NEW_GC = dict(zip(GC_JET_cols, GC_JET_COLS))
GC_JET.rename(columns=NEW_GC,inplace=True)
GC_JET['M1_M2'] = GC_JET['GC_JET_1'] - GC_JET['GC_JET_2']
GC_JET['M1_M3'] = GC_JET['GC_JET_1'] - GC_JET['GC_JET_3']
GC_JET['M2_M3'] = GC_JET['GC_JET_2'] - GC_JET['GC_JET_3']

In [15]:
###### JET #####
# LA
LA_JET = MY_BBG.filter(regex=LA_JET)
LA_JET_cols = LA_JET.columns.to_list()
LA_JET_COLS = ['LA_JET_1','LA_JET_2','LA_JET_3']
NEW_LA = dict(zip(LA_JET_cols, LA_JET_COLS))
LA_JET.rename(columns=NEW_LA,inplace=True)
LA_JET['M1_M2'] = LA_JET['LA_JET_1'] - LA_JET['LA_JET_2']
LA_JET['M1_M3'] = LA_JET['LA_JET_1'] - LA_JET['LA_JET_3']
LA_JET['M2_M3'] = LA_JET['LA_JET_2'] - LA_JET['LA_JET_3']

In [16]:
###### JET - DISTY #####
# la
LA_SPRD = MY_BBG.filter(regex=LA_SPRD)
LA_SPRD_cols = LA_SPRD.columns.to_list()
LA_SPRD_cols = LA_SPRD.columns.to_list()
LA_SPRD_COLS = ['LA_SPRD_1','LA_SPRD_2','LA_SPRD_3']
NEW_LA = dict(zip(LA_SPRD_cols, LA_SPRD_COLS))
LA_SPRD.rename(columns=NEW_LA,inplace=True)
LA_SPRD['SP_M1_M2'] = LA_SPRD['LA_SPRD_1'] - LA_SPRD['LA_SPRD_2']
LA_SPRD['SP_M1_M3'] = LA_SPRD['LA_SPRD_1'] - LA_SPRD['LA_SPRD_3']
LA_SPRD['SP_M2_M3'] = LA_SPRD['LA_SPRD_2'] - LA_SPRD['LA_SPRD_3']
# ny 
NY_SPRD = MY_BBG.filter(regex=NY_SPRD)
NY_SPRD_cols = NY_SPRD.columns.to_list()
NY_SPRD_COLS = ['NY_SPRD_1','NY_SPRD_2','NY_SPRD_3']
NEW_NY = dict(zip(NY_SPRD_cols, NY_SPRD_COLS))
NY_SPRD.rename(columns=NEW_NY,inplace=True)
NY_SPRD['SP_M1_M2'] = NY_SPRD['NY_SPRD_1'] - NY_SPRD['NY_SPRD_2']
NY_SPRD['SP_M1_M3'] = NY_SPRD['NY_SPRD_1'] - NY_SPRD['NY_SPRD_3']
NY_SPRD['SP_M2_M3'] = NY_SPRD['NY_SPRD_2'] - NY_SPRD['NY_SPRD_3']
# gc
GC_SPRD = MY_BBG.filter(regex=GC_SPRD)
GC_SPRD_cols = GC_SPRD.columns.to_list()
GC_SPRD_COLS = ['GC_SPRD_1','GC_SPRD_2','GC_SPRD_3']
NEW_GC = dict(zip(GC_SPRD_cols, GC_SPRD_COLS))
GC_SPRD.rename(columns=NEW_GC,inplace=True)
GC_SPRD['SP_M1_M2'] = GC_SPRD['GC_SPRD_1'] - GC_SPRD['GC_SPRD_2']
GC_SPRD['SP_M1_M3'] = GC_SPRD['GC_SPRD_1'] - GC_SPRD['GC_SPRD_3']
GC_SPRD['SP_M2_M3'] = GC_SPRD['GC_SPRD_2'] - GC_SPRD['GC_SPRD_3']

In [17]:
######CRUDE #####
CL = MY_BBG.filter(regex='CL')
CL_cols = CL.columns.to_list()
CL_COLS = ['CL_1','CL_2','CL_3']
NEW_CL = dict(zip(CL_cols, CL_COLS))
CL.rename(columns=NEW_CL,inplace=True)
CL['CL_M1_M2'] = CL['CL_1'] - CL['CL_2']
CL['CL_M1_M3'] = CL['CL_1'] - CL['CL_3']
CL['CL_M2_M3'] = CL['CL_2'] - CL['CL_3']
######DISTY #####
HO = MY_BBG.filter(regex="HO")
HO_cols = HO.columns.to_list()
HO_COLS = ["HO_1","HO_2","HO_3"]
NEW_HO = dict(zip(HO_cols, HO_COLS))
HO.rename(columns=NEW_HO,inplace=True)
HO["HO_M1_M2"] = HO["HO_1"] - HO["HO_2"]
HO["HO_M1_M3"] = HO["HO_1"] - HO["HO_3"]
HO["HO_M2_M3"] = HO["HO_2"] - HO["HO_3"]
###### put it all together

In [18]:
ALL_PRICES = pd.concat([LA_JET,GC_JET,NY_JET,LA_SPRD,GC_SPRD,NY_SPRD,CL,HO], axis=1)

In [19]:
ALL_PRICES.head()

,LA_JET_1,LA_JET_2,LA_JET_3,M1_M2,M1_M3,M2_M3,GC_JET_1,GC_JET_2,GC_JET_3,M1_M2,...,CL_3,CL_M1_M2,CL_M1_M3,CL_M2_M3,HO_1,HO_2,HO_3,HO_M1_M2,HO_M1_M3,HO_M2_M3
week_ending,,,,,,,,,,,,,,,,,,,,,
2011-01-07,258.36,259.72,261.78,-1.36,-3.42,-2.06,254.54,256.16,257.00,-1.62,...,91.58,-1.12,-2.05,-0.93,251.98,252.68,252.37,-0.70,-0.39,0.31
2011-01-14,266.11,268.41,272.04,-2.30,-5.93,-3.63,262.47,263.77,264.36,-1.30,...,93.11,-1.11,-2.08,-0.97,260.76,261.06,260.35,-0.30,0.41,0.71
2011-01-21,267.50,269.49,273.28,-1.99,-5.78,-3.79,265.68,266.68,266.90,-1.00,...,92.33,-1.04,-2.28,-1.24,264.40,264.72,263.66,-0.32,0.74,1.06
2011-01-28,268.82,270.86,274.52,-2.04,-5.70,-3.66,266.45,267.57,268.01,-1.12,...,91.09,-2.08,-3.82,-1.74,264.54,264.81,264.01,-0.27,0.53,0.80
2011-02-04,280.59,284.18,285.28,-3.59,-4.69,-1.10,277.86,278.68,279.75,-0.82,...,95.28,-2.63,-4.60,-1.97,275.37,275.12,275.02,0.25,0.35,0.10


In [22]:
ALL_PRICES['LA_NY_1'] = ALL_PRICES['LA_JET_1'] - ALL_PRICES['NY_JET_1']
ALL_PRICES['LA_GC_1'] = ALL_PRICES['LA_JET_1'] - ALL_PRICES['GC_JET_1']
ALL_PRICES['NY_GC_1'] = ALL_PRICES['NY_JET_1'] - ALL_PRICES['GC_JET_1']

,LA_JET_1,LA_JET_2,LA_JET_3,M1_M2,M1_M3,M2_M3,GC_JET_1,GC_JET_2,GC_JET_3,M1_M2,...,CL_M2_M3,HO_1,HO_2,HO_3,HO_M1_M2,HO_M1_M3,HO_M2_M3,LA_NY_1,LA_GC_1,NY_GC_1
week_ending,,,,,,,,,,,,,,,,,,,,,
2011-01-07,258.36,259.72,261.78,-1.36,-3.42,-2.06,254.54,256.16,257.00,-1.62,...,-0.93,251.98,252.68,252.37,-0.70,-0.39,0.31,3.82,3.82,0.0
2011-01-14,266.11,268.41,272.04,-2.30,-5.93,-3.63,262.47,263.77,264.36,-1.30,...,-0.97,260.76,261.06,260.35,-0.30,0.41,0.71,3.64,3.64,0.0
2011-01-21,267.50,269.49,273.28,-1.99,-5.78,-3.79,265.68,266.68,266.90,-1.00,...,-1.24,264.40,264.72,263.66,-0.32,0.74,1.06,1.82,1.82,0.0
2011-01-28,268.82,270.86,274.52,-2.04,-5.70,-3.66,266.45,267.57,268.01,-1.12,...,-1.74,264.54,264.81,264.01,-0.27,0.53,0.80,2.37,2.37,0.0
2011-02-04,280.59,284.18,285.28,-3.59,-4.69,-1.10,277.86,278.68,279.75,-0.82,...,-1.97,275.37,275.12,275.02,0.25,0.35,0.10,2.73,2.73,0.0


In [23]:
ALL_PRICES['LA_NY_2'] = ALL_PRICES['LA_JET_2'] - ALL_PRICES['NY_JET_2']
ALL_PRICES['LA_GC_2'] = ALL_PRICES['LA_JET_2'] - ALL_PRICES['GC_JET_2']
ALL_PRICES['NY_GC_2'] = ALL_PRICES['NY_JET_2'] - ALL_PRICES['GC_JET_2']

In [24]:
ALL_PRICES['LA_NY_3'] = ALL_PRICES['LA_JET_3'] - ALL_PRICES['NY_JET_3']
ALL_PRICES['LA_GC_3'] = ALL_PRICES['LA_JET_3'] - ALL_PRICES['GC_JET_3']
ALL_PRICES['NY_GC_3'] = ALL_PRICES['NY_JET_3'] - ALL_PRICES['GC_JET_3']

In [25]:
ALL_PRICES.tail()

,LA_JET_1,LA_JET_2,LA_JET_3,M1_M2,M1_M3,M2_M3,GC_JET_1,GC_JET_2,GC_JET_3,M1_M2,...,HO_M2_M3,LA_NY_1,LA_GC_1,NY_GC_1,LA_NY_2,LA_GC_2,NY_GC_2,LA_NY_3,LA_GC_3,NY_GC_3
week_ending,,,,,,,,,,,,,,,,,,,,,
2020-05-08,87.91,66.41,79.15,21.50,8.76,-12.74,60.77,66.03,72.53,-5.26,...,-4.03,20.14,27.14,7.00,-6.70,0.38,7.08,2.43,6.62,4.19
2020-05-15,73.32,69.56,81.93,3.76,-8.61,-12.37,64.06,68.58,74.10,-4.52,...,-3.53,0.48,9.26,8.78,-8.63,0.98,9.61,0.39,7.83,7.44
2020-05-22,88.50,81.57,93.77,6.93,-5.27,-12.20,86.24,89.54,92.35,-3.30,...,-3.40,2.00,2.26,0.26,-8.99,-7.97,1.02,0.05,1.42,1.37
2020-05-29,89.28,79.75,91.91,9.53,-2.63,-12.16,84.20,87.23,90.17,-3.03,...,-3.46,4.36,5.08,0.72,-8.89,-7.48,1.41,0.29,1.74,1.45
2020-06-05,97.72,97.70,100.10,0.02,-2.38,-2.40,93.82,96.13,98.12,-2.31,...,-2.72,2.93,3.90,0.97,-0.17,1.57,1.74,-0.16,1.98,2.14


In [27]:
ALL_PRICES.to_csv('prices_spreads.csv')

In [ ]:
##### here we "AIMED" to change the headers back to usual
m1.rename(columns=lambda x: x[0:5], inplace=True)
m2.rename(columns=lambda x: x[0:5], inplace=True)
m3.rename(columns=lambda x: x[0:5], inplace=True)
spread_m1_m2 = m1.subtract(m2)          
spread_m1_m3 = m1.subtract(m3)
spread_m2_m3 = m2.subtract(m3)
bbg_my_sub1 = bbg_my[['short_name']]
bbg_my_sub2 = bbg_my[['long_name']]
bbg_my_sub3 = bbg_my[['unit']]
# nng_my_sub['ticker'] = MY_BBG.index
short_name_dict = dict(zip(bbg_my_sub1.index, bbg_my_sub1.short_name))
long_name_dict = dict(zip(bbg_my_sub2.index, bbg_my_sub2.long_name))
unit_name_dict = dict(zip(bbg_my_sub3.index, bbg_my_sub3.unit))

# GET each subset's keys too so can replace
keys_to_extract = m1_cols
m1_dict_s = {key: short_name_dict[key] for key in keys_to_extract}
m1_dict_l = {key: long_name_dict[key] for key in keys_to_extract}
m1_dict_u = {key: unit_name_dict[key] for key in keys_to_extract}

In [ ]:
spread_m1_m2 = m1.subtract(m2)          
spread_m1_m3 = m1.subtract(m3)
spread_m2_m3 = m2.subtract(m3)
bbg_my_sub1 = bbg_my[['short_name']]
bbg_my_sub2 = bbg_my[['long_name']]
bbg_my_sub3 = bbg_my[['unit']]
# nng_my_sub['ticker'] = MY_BBG.index
short_name_dict = dict(zip(bbg_my_sub1.index, bbg_my_sub1.short_name))
long_name_dict = dict(zip(bbg_my_sub2.index, bbg_my_sub2.long_name))
unit_name_dict = dict(zip(bbg_my_sub3.index, bbg_my_sub3.unit))
# GET each subset's keys too so can replace
keys_to_extract = m1_cols
m1_dict_s = {key: short_name_dict[key] for key in keys_to_extract}
m1_dict_l = {key: long_name_dict[key] for key in keys_to_extract}
m1_dict_u = {key: unit_name_dict[key] for key in keys_to_extract}
# need to pick the subset of each month and create a dict

In [ ]:
# m1.rename(columns=m1_dict_l, inplace=True) keep this
# m1.rename(columns=lambda x: x[0:5], inplace=True)
# m2.rename(columns=lambda x: x[0:5], inplace=True)
# m3.rename(columns=lambda x: x[0:5], inplace=True)